In [ ]:
# import pandas as pd
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# from numpy import array
# from numpy import hstack
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# import pymysql
# from datetime import datetime, timedelta
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
# import seaborn as sns
# from statsmodels.tsa.seasonal import seasonal_decompose
# from sklearn.metrics import mean_squared_error
# from math import sqrt
# from typing import List
# from typing import Optional
# from sqlalchemy import ForeignKey
# from sqlalchemy import String
# from sqlalchemy.orm import DeclarativeBase
# from sqlalchemy.orm import Mapped
# from sqlalchemy.orm import mapped_column
# from sqlalchemy.orm import relationship
# import random, sqlite3


In [ ]:
# from datetime import datetime
# print(f"Running Water_Elevation.ipynb at: {datetime.now()}")
# %run Water_Elevation.ipynb

In [ ]:
# import time
# from IPython import get_ipython

# def run_notebook():
#     while True:
#         get_ipython().run_line_magic('run', 'Water_Elevation.ipynb')
#         print("Notebook executed at:", time.strftime("%Y-%m-%d %H:%M:%S"))
#         time.sleep(300)  # 300 seconds = 5 minutes

# # Start the loop
# run_notebook()


In [ ]:
# # Database connection
# db_name = "floodeck_alpha"
# db_host = "localhost"
# db_username = "root"
# db_password = ""

# try:
#     conn = pymysql.connect(host=db_host,
#                            port=int(3306),
#                            user=db_username,
#                            password=db_password,
#                            db=db_name)
#     print("Connection successful")
# except Exception as e:
#     print(e)

In [ ]:
# import time
# import threading
# from IPython import get_ipython

# def run_notebook_periodically():
#     while True:
#         get_ipython().run_line_magic('run', 'Sensor_A_Data_Send.ipynb')
#         print("Notebook executed at:", time.strftime("%Y-%m-%d %H:%M:%S"))
#         time.sleep(60)  # 300 seconds = 5 minutes

# # Create and start a thread to run the notebook periodically
# execution_thread = threading.Thread(target=run_notebook_periodically)
# execution_thread.daemon = True  # This ensures the thread will not prevent the program from exiting
# execution_thread.start()


In [ ]:
# import time
# import threading
# from IPython import get_ipython

# def run_notebook_periodically():
#     while True:
#         get_ipython().run_line_magic('run', 'Sensor_B_Data_Send.ipynb')
#         print("Notebook executed at:", time.strftime("%Y-%m-%d %H:%M:%S"))
#         time.sleep(60)  # 300 seconds = 5 minutes

# # Create and start a thread to run the notebook periodically
# execution_thread = threading.Thread(target=run_notebook_periodically)
# execution_thread.daemon = True  # This ensures the thread will not prevent the program from exiting
# execution_thread.start()


In [ ]:
import time
import threading
from IPython import get_ipython

def run_notebook_periodically():
    while True:
        get_ipython().run_line_magic('run', 'Water_Elevation.ipynb')
        print("Notebook executed at:", time.strftime("%Y-%m-%d %H:%M:%S"))
        time.sleep(60)  # 300 seconds = 5 minutes

# Create and start a thread to run the notebook periodically
execution_thread = threading.Thread(target=run_notebook_periodically)
execution_thread.daemon = True  # This ensures the thread will not prevent the program from exiting
execution_thread.start()


In [ ]:
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes, JobQueue

import nest_asyncio
import asyncio

DANGER = "DANGER"

# Global variable to store the chat ID
chat_id = None

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

# Your bot token here
TOKEN = '7167249563:AAFtTw4yC-fFnMXrsmJIcQacbt4Tt5tupDw'

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    global chat_id
    chat_id = update.effective_chat.id
    await update.message.reply_text('Start the services')

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /help is issued."""
    await update.message.reply_text('Use any command available')

#Point A and Point B now
async def water_point(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /water_point is issued."""
    await update.message.reply_text(f'Water height of Point A and Point B: \n{sensor_a_scan_water_height}, {point_a_status}, \n{sensor_b_scan_water_height}, {point_b_status}, \n{latest_timestamp}')

#Prediction of Point A and Point B now
async def prediction_water_point(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /prediction_water_point is issued."""
    await update.message.reply_text(f'Prediction water height of Point A and Point B: \n{predicted_value_a.iloc[0]}, \n{predicted_value_b.iloc[0]}, \n{future_timestamp.iloc[0]}')

#Water Elevation 
async def water_elevation(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /water_elevation is issued."""
    await update.message.reply_text(f'Water elevation: {Hx}, \n{elevation_status}, \n{latest_timestamp}')

#Water Predicted Elevation
async def predicted_water_elevation(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /predicted_water_elevation is issued."""
    await update.message.reply_text(f'Predicted water elevation prediction: {predicted_Hx}, \n{elevation_status}, \n{latest_timestamp}')

async def check_danger_status(context: ContextTypes.DEFAULT_TYPE) -> None:
    """Check if any point is in DANGER status and send a message."""
    global chat_id
    if chat_id is not None and (point_a_status == DANGER or point_b_status == DANGER or elevation_status == DANGER):
        danger_message = (f'ALERT! Danger status detected: \n'
                          f'Point A: {sensor_a_scan_water_height}, {point_a_status}\n'
                          f'Point B: {sensor_b_scan_water_height}, {point_b_status}\n'
                          f'Water Elevation: {Hx}, {elevation_status}\n'
                          f'{latest_timestamp}')
        await context.bot.send_message(chat_id=chat_id, text=danger_message)

async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Echo the user message."""
    user_text = update.message.text
    await update.message.reply_text(user_text)

async def main() -> None:
    """Start the bot."""
    application = ApplicationBuilder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("water_point", water_point))
    application.add_handler(CommandHandler("prediction_water_point", prediction_water_point))
    application.add_handler(CommandHandler("water_elevation", water_elevation))
    application.add_handler(CommandHandler("predicted_water_elevation", predicted_water_elevation))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

    # Run the check_danger_status function periodically (e.g., every minute)
    job_queue = application.job_queue
    job_queue.run_repeating(check_danger_status, interval=60, first=0)
    
    await application.initialize()
    await application.start()
    await application.updater.start_polling()


if __name__ == '__main__':
    nest_asyncio.apply()
    asyncio.run(main())


In [ ]:
import pandas as pd
import pymysql

In [ ]:
# Database connection
db_name = "floodeck_alpha"
db_host = "localhost"
db_username = "root"
db_password = ""

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    print("Connection successful")
except Exception as e:
    print(e)

In [ ]:
import time
import threading
import requests
import schedule
import pandas as pd
import sqlite3

# Assuming conn is a valid database connection
# conn = sqlite3.connect('your_database.db')

def fetch_subscribers():
    query = "SELECT phone_number, whatsapp_api FROM whatsapp_subscribers"
    return pd.read_sql_query(query, conn)

def send_whatsapp_message(phone_number, message, apikey, retries=3):
    url = f"https://api.callmebot.com/whatsapp.php?phone={phone_number}&text={message}&apikey={apikey}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.status_code, response.text
    except requests.RequestException as e:
        if retries > 0:
            time.sleep(60)  # Wait for 5 seconds before retrying
            return send_whatsapp_message(phone_number, message, apikey, retries-1)
        else:
            return None, str(e)

def check_and_send_messages():
    # Ensure these variables are defined and updated appropriately
    global point_a_status, point_b_status, elevation_status
    global sensor_a_scan_water_height, sensor_b_scan_water_height, Hx, latest_timestamp

    if point_a_status == "DANGER" or point_b_status == "DANGER" or elevation_status == "DANGER":
        danger_message = (f'ALERT! Danger status detected: \n'
                          f'Point A: {sensor_a_scan_water_height}, {point_a_status}\n'
                          f'Point B: {sensor_b_scan_water_height}, {point_b_status}\n'
                          f'Water Elevation: {Hx}, {elevation_status}\n'
                          f'{latest_timestamp}')
        message = danger_message

        # Fetch the latest subscribers
        subscribers_df = fetch_subscribers()

        # Send messages to all subscribers
        for index, row in subscribers_df.iterrows():
            phone_number = row['phone_number']
            apikey = row['whatsapp_api']
            status_code, response_text = send_whatsapp_message(phone_number, message, apikey)
            print(f"Phone Number: {phone_number}")
            print(f"Status Code: {status_code}")
            print(f"Response Text: {response_text}\n")

def schedule_checker():
    while True:
        schedule.run_pending()
        time.sleep(5)

# Schedule the task to run every minute
schedule.every(5).minutes.do(check_and_send_messages)

# Start the background thread
thread = threading.Thread(target=schedule_checker)
thread.daemon = True
thread.start()
